In [1]:
import pandas as pd
# pd.set_option("display.max_colwidth", 200)
import numpy as np
import json
import re
import gzip
import spacy

import gensim
from gensim import corpora
import os.path
from gensim import corpora
from gensim.models import LsiModel
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\tanma\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from nltk.corpus import stopwords

import nltk

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer

from nltk.stem import WordNetLemmatizer

from tqdm import tqdm

In [3]:
review_df = pd.read_csv('./data/reviews.csv')
review_df.drop(columns=['Unnamed: 0'], inplace= True)

review_df['user_id'].nunique()

review_df['review_text'].fillna('', inplace = True)

In [4]:
review_df.head()

engaged_df = review_df[(review_df['n_votes'] > 0)|(review_df['n_comments'] > 0)]

engaged_df.head()

,book_id,user_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
6,5805,4cbecbc15af3db041a8e0f594c642bb5,58f2301bd2d4bbfc1b51e4e5fb161cfe,5,"Remember, remember, the fifth of November. Thi...",Wed Jun 13 17:55:53 -0700 2012,Mon Jan 30 05:58:01 -0800 2017,Thu Jun 14 00:00:00 -0700 2012,NaN,7,0
8,5805,49cc59f1c479d698507627b401d47ecf,761a17f52538341a085b629a316204a1,4,Tinha apontado este livro como um dos que tinh...,Sun Jan 05 03:27:10 -0800 2014,Tue Oct 28 15:58:20 -0700 2014,Tue Oct 28 15:58:20 -0700 2014,NaN,1,0
10,5805,5f03864c758bfceb6d7d5e93eeb20044,ac54c3ce0c9f660c03881b0668f79c60,5,Review coming soon! www.youtube.com/ReadTomes,Tue Feb 12 11:38:36 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 12:00:47 -0800 2013,Tue Feb 12 00:00:00 -0800 2013,1,0
12,5805,c309dff1695ed8558b29ea8dcd7479b8,0da0bcc469c2acd15350f9a8f0a74e2b,5,"What better way to celebrate Guy Fawkes Day, t...",Tue May 20 09:52:36 -0700 2014,Thu Mar 16 09:49:11 -0700 2017,Wed Nov 05 14:22:10 -0800 2014,Wed Nov 05 00:00:00 -0800 2014,2,0
13,5805,0f6b8c04f811e05c8978bd6b66ce7685,7d670b6c8cac0c086e21ae2f1af6eccb,4,To note - I am writing this review a quarter c...,Tue Jul 03 16:43:26 -0700 2012,Fri Feb 27 14:46:00 -0800 2015,Tue Jan 01 00:00:00 -0800 1991,NaN,10,0


In [5]:
lm  = WordNetLemmatizer()

# nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])



In [6]:
stop_words = set(stopwords.words('english'))

words = set(nltk.corpus.words.words())

In [7]:
def preprocess(x):
    try:
        x = re.sub("n\'t", " not", x)
        x = re.sub("\'d", " would", x)
        x = re.sub("[^a-zA-Z0-9#]", " ", x)
        x = re.sub("www", " ", x)
        x = re.sub("http", " ", x)
        x = re.sub("https", " ", x)
        x = re.sub("com", " ", x)
        x = re.sub("url", " ", x)
        return x
    except:
        print(x)
        return ""
    

def lemmas(x, lm, tags=['NOUN', 'ADJ']):
    x = word_tokenize(x)
    
    x = [lm.lemmatize(w).lower() for w in x]
    
#     print(x)
    x = nlp(" ".join(x))
#     x = [w for w in x if w in words]
    x = [token.lemma_ for token in x if token.pos_ in tags or token.lemma_ in words]
    x = [w for w in x if not w.lower() in stop_words]
    x = [w for w in x if len(w) > 2]
    
    return x

In [8]:
engaged_df['clean_reviews'] = engaged_df['review_text'].apply(lambda x: preprocess(x))



C:\Users\tanma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [9]:
%%time
engaged_df['clean_reviews'] = engaged_df['clean_reviews'].apply(lambda x: lemmas(x, lm))

Wall time: 5min 32s


C:\Users\tanma\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
dictionary = corpora.Dictionary(engaged_df['clean_reviews'])

doc_term_matrix = tqdm([dictionary.doc2bow(rev) for rev in engaged_df['clean_reviews']])

  0%|                                                                                        | 0/39787 [00:00<?, ?it/s]

In [21]:
def compute_coherence_values(dictionary, doc_term_matrix, doc_clean, stop, start=2, step=1):
    """
    Input   : dictionary : Gensim dictionary
              corpus : Gensim corpus
              texts : List of input texts
              stop : Max num of topics
    purpose : Compute c_v coherence for various number of topics
    Output  : model_list : List of LSA topic models
              coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, stop, step)):
        # generate LSA model
        model = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=doc_clean, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [25]:
model_list, coherence_values = compute_coherence_values(dictionary, doc_term_matrix, engaged_df['clean_reviews'], 12)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:49<00:00, 22.99s/it]


### Training the best model

In [12]:
%%time
lsi = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)

100%|█████████████████████████████████████████████████████████████████████████| 39787/39787 [00:03<00:00, 11458.06it/s]


Wall time: 5.75 s


In [13]:
lsi.show_topics()

[(0,
  '0.309*"story" + 0.280*"book" + 0.239*"like" + 0.238*"read" + 0.218*"one" + 0.194*"character" + 0.171*"get" + 0.150*"really" + 0.145*"make" + 0.139*"love"'),
 (1,
  '0.760*"book" + -0.354*"volume" + 0.233*"read" + -0.222*"story" + -0.208*"character" + -0.157*"series" + 0.107*"novel" + 0.102*"graphic" + -0.093*"get" + -0.073*"love"'),
 (2,
  '0.674*"batman" + 0.504*"story" + -0.204*"volume" + 0.171*"joker" + -0.150*"like" + -0.134*"love" + -0.130*"series" + -0.128*"read" + -0.104*"get" + 0.100*"dark"'),
 (3,
  '-0.595*"story" + 0.340*"batman" + -0.294*"novel" + -0.270*"graphic" + 0.267*"like" + -0.224*"read" + 0.142*"get" + -0.138*"love" + 0.112*"one" + 0.092*"know"'),
 (4,
  '-0.517*"read" + -0.456*"batman" + 0.341*"story" + -0.247*"volume" + -0.232*"novel" + -0.213*"graphic" + -0.167*"love" + 0.154*"book" + -0.129*"series" + 0.102*"like"')]